In [1]:
import pandas as pd
import re
import transformers as ppb
import numpy as np
from scipy import sparse

In [102]:
DATA_DIR = "./data"
admissions_file = DATA_DIR + '/ADMISSIONS.csv'
diagnoses_file = DATA_DIR + '/DIAGNOSES_ICD.csv'

First use admissions records to create READMIT label and STAY_LENGTH

In [103]:
full_admissions = pd.read_csv(admissions_file)
print(full_admissions.shape)
print(full_admissions.dtypes)
full_admissions.head()

(58976, 19)
ROW_ID                   int64
SUBJECT_ID               int64
HADM_ID                  int64
ADMITTIME               object
DISCHTIME               object
DEATHTIME               object
ADMISSION_TYPE          object
ADMISSION_LOCATION      object
DISCHARGE_LOCATION      object
INSURANCE               object
LANGUAGE                object
RELIGION                object
MARITAL_STATUS          object
ETHNICITY               object
EDREGTIME               object
EDOUTTIME               object
DIAGNOSIS               object
HOSPITAL_EXPIRE_FLAG     int64
HAS_CHARTEVENTS_DATA     int64
dtype: object


,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [104]:
# convert all to datetime
full_admissions['ADMITTIME'] = pd.to_datetime(full_admissions['ADMITTIME'])
full_admissions['DISCHTIME'] = pd.to_datetime(full_admissions['DISCHTIME'])
full_admissions['DEATHTIME'] = pd.to_datetime(full_admissions['DEATHTIME'])
full_admissions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaT,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [105]:
# order by subject and discharge 
admits_patient_df = full_admissions.sort_values(by=['SUBJECT_ID','DISCHTIME']).loc[:,('ROW_ID','SUBJECT_ID','HADM_ID','ADMITTIME',\
    'DISCHTIME','DEATHTIME','ADMISSION_TYPE','DIAGNOSIS','HOSPITAL_EXPIRE_FLAG','HAS_CHARTEVENTS_DATA')]
admits_patient_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
211,1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaT,NEWBORN,NEWBORN,0,1
212,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,EMERGENCY,HYPOTENSION,0,1
213,3,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,EMERGENCY,"FEVER,DEHYDRATION,FAILURE TO THRIVE",0,1
214,4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaT,NEWBORN,NEWBORN,0,1
215,5,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,ELECTIVE,CHRONIC RENAL FAILURE/SDA,0,1


In [111]:
# separate only patients with more than one admission to create READMIT flag
mult_admits = admits_patient_df[admits_patient_df.duplicated(subset=['SUBJECT_ID'],keep=False)]
print(mult_admits.shape)
mult_admits.head()

(19993, 10)


,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
224,14,17,194023,2134-12-27 07:15:00,2134-12-31 16:05:00,NaT,ELECTIVE,PATIENT FORAMEN OVALE\ PATENT FORAMEN OVALE MI...,0,1
225,15,17,161087,2135-05-09 14:11:00,2135-05-13 14:40:00,NaT,EMERGENCY,PERICARDIAL EFFUSION,0,1
229,19,21,109451,2134-09-11 12:17:00,2134-09-24 16:15:00,NaT,EMERGENCY,CONGESTIVE HEART FAILURE,0,1
230,20,21,111970,2135-01-30 20:50:00,2135-02-08 02:08:00,2135-02-08 02:08:00,EMERGENCY,SEPSIS,1,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaT,ELECTIVE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1


In [84]:
# showing what the rolling function does on a single example with 3 admissions
mult_admits[mult_admits['SUBJECT_ID']==36].loc[:,('DISCHTIME','READMIT')].rolling(window='30D', on='DISCHTIME',closed='left').sum()

,DISCHTIME,READMIT
15,2131-05-08 14:00:00,NaN
16,2131-05-25 13:30:00,1.0
17,2134-05-20 13:16:00,NaN


In [112]:
# identify readmits within 30 days
subjects = mult_admits['SUBJECT_ID'].unique()
mult_admits.loc[:,'READMIT_2'] = 1

# assign flag for each patient if they were re-admitted
for subject in subjects:
    readmitted = mult_admits[mult_admits['SUBJECT_ID'] == subject].loc[:,('DISCHTIME','READMIT_2')].rolling(window='30D', on='DISCHTIME',closed='left').sum()
    #print(readmitted)
    mult_admits.loc[readmitted.index, 'READMIT_2'] = readmitted['READMIT_2']
    #print(subject)

# assign READMIT flag to the occurrence PRIOR to the readmit
mult_admits['READMIT'] = mult_admits['READMIT_2'].shift(-1)

/home/npodpx/venv/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/home/npodpx/venv/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/tmp/ipykernel_23941/4144693818.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [109]:
# confirm READMIT working as expected
mult_admits[mult_admits['SUBJECT_ID'] == 36]

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,READMIT_2,READMIT
15,36,36,182104,2131-04-30 07:15:00,2131-05-08 14:00:00,NaT,EMERGENCY,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,NaN,1.0
16,37,36,122659,2131-05-12 19:49:00,2131-05-25 13:30:00,NaT,EMERGENCY,CHEST PAIN/SHORTNESS OF BREATH,0,1,1.0,NaN
17,38,36,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaT,ELECTIVE,VENTRAL HERNIA/SDA,0,1,NaN,NaN


In [115]:
single_admits = admits_patient_df[~admits_patient_df.duplicated(subset=['SUBJECT_ID'],keep=False)]
print(single_admits.shape)
print('total rows in single and mult admits tables:',mult_admits.shape[0] + single_admits.shape[0])

(38983, 10)
total rows in single and mult admits tables: 58976


In [150]:
# concatenate individuals with multiple admissions and single admissions back to full dataset
full_admits_labels = pd.concat([mult_admits, single_admits])
full_admits_labels['READMIT'] = full_admits_labels['READMIT'].fillna(0)
full_admits_labels['READMIT'] = np.where(full_admits_labels['READMIT'] > 0, 1, 0)
full_admits_labels.drop(columns=['READMIT_2'],inplace=True)
print(full_admits_labels.shape)
full_admits_labels.head()

(58976, 11)


,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,READMIT
224,14,17,194023,2134-12-27 07:15:00,2134-12-31 16:05:00,NaT,ELECTIVE,PATIENT FORAMEN OVALE\ PATENT FORAMEN OVALE MI...,0,1,0.0
225,15,17,161087,2135-05-09 14:11:00,2135-05-13 14:40:00,NaT,EMERGENCY,PERICARDIAL EFFUSION,0,1,0.0
229,19,21,109451,2134-09-11 12:17:00,2134-09-24 16:15:00,NaT,EMERGENCY,CONGESTIVE HEART FAILURE,0,1,0.0
230,20,21,111970,2135-01-30 20:50:00,2135-02-08 02:08:00,2135-02-08 02:08:00,EMERGENCY,SEPSIS,1,1,0.0
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaT,ELECTIVE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,0.0


Next, create Stay Length both as a timedelta and as an integer of seconds

In [127]:
full_admits_labels.loc[:,('STAY_LENGTH_TD')] = full_admits_labels.loc[:,('DISCHTIME')] - full_admits_labels.loc[:,('ADMITTIME')]
full_admits_labels.loc[:,('STAY_LENGTH_SEC')] = full_admits_labels.loc[:,('STAY_LENGTH_TD')].dt.seconds
full_admits_labels.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,READMIT,STAY_LENGTH_TD,STAY_LENGTH_SEC
224,14,17,194023,2134-12-27 07:15:00,2134-12-31 16:05:00,NaT,ELECTIVE,PATIENT FORAMEN OVALE\ PATENT FORAMEN OVALE MI...,0,1,0.0,4 days 08:50:00,31800
225,15,17,161087,2135-05-09 14:11:00,2135-05-13 14:40:00,NaT,EMERGENCY,PERICARDIAL EFFUSION,0,1,0.0,4 days 00:29:00,1740
229,19,21,109451,2134-09-11 12:17:00,2134-09-24 16:15:00,NaT,EMERGENCY,CONGESTIVE HEART FAILURE,0,1,0.0,13 days 03:58:00,14280
230,20,21,111970,2135-01-30 20:50:00,2135-02-08 02:08:00,2135-02-08 02:08:00,EMERGENCY,SEPSIS,1,1,0.0,8 days 05:18:00,19080
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaT,ELECTIVE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,0.0,5 days 11:55:00,42900


Finally, attach diagnosis ICD-9 code from DIAGNOSES_ICD table

In [123]:
diagnoses = pd.read_csv(diagnoses_file)
print(diagnoses.shape)
print(diagnoses.dtypes)
diagnoses.head()

(651047, 5)
ROW_ID          int64
SUBJECT_ID      int64
HADM_ID         int64
SEQ_NUM       float64
ICD9_CODE      object
dtype: object


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [137]:
diagnoses['ICD9_CODE'].sort_values()

304974    0030
505370    0030
519876    0031
594655    0038
414953    0038
          ... 
333526     NaN
356738     NaN
360813     NaN
386777     NaN
549256     NaN
Name: ICD9_CODE, Length: 651047, dtype: object

In [138]:
# only select the top priority ICD diagnosis to attempt to predict
diagnoses_priority = diagnoses[diagnoses['SEQ_NUM'] == 1]
diagnoses_priority.drop(columns=['ROW_ID','SEQ_NUM'],inplace=True)

/home/npodpx/venv/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [139]:
full_labels = full_admits_labels.merge(diagnoses_priority, on=['SUBJECT_ID','HADM_ID'])
print(full_labels.shape)
full_labels.head()

(58929, 14)


,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,READMIT,STAY_LENGTH_TD,STAY_LENGTH_SEC,ICD9_CODE
0,14,17,194023,2134-12-27 07:15:00,2134-12-31 16:05:00,NaT,ELECTIVE,PATIENT FORAMEN OVALE\ PATENT FORAMEN OVALE MI...,0,1,0.0,4 days 08:50:00,31800,7455
1,15,17,161087,2135-05-09 14:11:00,2135-05-13 14:40:00,NaT,EMERGENCY,PERICARDIAL EFFUSION,0,1,0.0,4 days 00:29:00,1740,4239
2,19,21,109451,2134-09-11 12:17:00,2134-09-24 16:15:00,NaT,EMERGENCY,CONGESTIVE HEART FAILURE,0,1,0.0,13 days 03:58:00,14280,41071
3,20,21,111970,2135-01-30 20:50:00,2135-02-08 02:08:00,2135-02-08 02:08:00,EMERGENCY,SEPSIS,1,1,0.0,8 days 05:18:00,19080,0388
4,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaT,ELECTIVE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,0.0,5 days 11:55:00,42900,41401


In [140]:
labels_final = full_labels.loc[:,('SUBJECT_ID','HADM_ID','READMIT','STAY_LENGTH_SEC','ICD9_CODE')]
labels_final.to_pickle('./data/labels_final_df.pkl')

Finally finally, create 3 arrays of labels to feed into models

In [141]:
# readmit array
readmit_np = np.array(labels_final['READMIT'])
sparse_readmit_labels = sparse.csr_matrix(readmit_np)
readmit_labels_file = DATA_DIR + '/sparse_readmit_labels'
sparse.save_npz(readmit_labels_file, sparse_readmit_labels)

In [142]:
# stay length array
staylen_np = np.array(labels_final['STAY_LENGTH_SEC'])
sparse_staylen_labels = sparse.csr_matrix(staylen_np)
staylen_labels_file = DATA_DIR + '/sparse_staylen_labels'
sparse.save_npz(staylen_labels_file, sparse_staylen_labels)

In [ ]:
# diagnoses array - CURRENTLY DOESN'T WORK B/C ICD-9 CODES NOT ALL NUMERIC!
#diagnoses_np = np.array(labels_final['ICD9_CODE'])
#sparse_diagnoses_labels = sparse.csr_matrix(diagnoses_np)
#diagnoses_labels_file = DATA_DIR + '/sparse_diagnoses_labels'
#sparse.save_npz(diagnoses_labels_file, sparse_diagnoses_labels)

In [145]:
best_note_df = pd.read_pickle('./data/best_note_df.pkl')

In [146]:
notes_labels = best_note_df.merge(labels_final, on=['SUBJECT_ID','HADM_ID'])
notes_labels.head()

,SUBJECT_ID,HADM_ID,TEXT,IS_DISCHARGE,IS_NURSING,IS_OTHER,READMIT,STAY_LENGTH_SEC,ICD9_CODE
0,22532,167853,['Admission Date: [**2151-7-16**] Disch...,1,0,0,0.0,16860,01193
1,13702,107527,['Admission Date: [**2118-6-2**] Discha...,1,0,0,0.0,71820,5191
2,13702,167118,['Admission Date: [**2119-5-4**] ...,1,0,0,0.0,22500,5191
3,13702,196489,"[""Admission Date: [**2124-7-21**] ...",1,0,0,0.0,1020,51884
4,26880,135453,['Admission Date: [**2162-3-3**] ...,1,0,0,0.0,70140,80506


In [147]:
readmit_np = np.array(notes_labels['READMIT'])
sparse_readmit_labels = sparse.csr_matrix(readmit_np)
readmit_labels_file = DATA_DIR + '/sparse_readmit_labels'
sparse.save_npz(readmit_labels_file, sparse_readmit_labels)

In [148]:
staylen_np = np.array(notes_labels['STAY_LENGTH_SEC'])
sparse_staylen_labels = sparse.csr_matrix(staylen_np)
staylen_labels_file = DATA_DIR + '/sparse_staylen_labels'
sparse.save_npz(staylen_labels_file, sparse_staylen_labels)

In [149]:
notes_labels.to_pickle('./data/notes_labels.pkl')